In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter, defaultdict
from sklearn.metrics import roc_curve, auc
from sklearn import preprocessing

In [29]:
dataset = pd.read_csv('network_backup_dataset.csv')

In [64]:
dataset.columns

Index(['Week #', 'Day of Week', 'Backup Start Time - Hour of Day',
       'Work-Flow-ID', 'File Name', 'Size of Backup (GB)',
       'Backup Time (hour)'],
      dtype='object')

In [76]:
def getX_Y_from_dataset(dataset):
    Y = list(dataset['Size of Backup (GB)'].as_matrix())
    X = dataset.drop(['Size of Backup (GB)'],axis=1)
    return X,Y

In [77]:
X,Y= getX_Y_from_dataset(dataset)

In [78]:
print(X.shape,len())

AttributeError: 'list' object has no attribute 'shape'

In [62]:
def one_hot_encode(dataset):
    le = preprocessing.LabelEncoder()
    dataset_cat = dataset.select_dtypes(include=[object])
    dataset_cat.head()
    dataset_cat = dataset_cat.apply(le.fit_transform)
    oh_enc = preprocessing.OneHotEncoder()
    oh_enc.fit(dataset_cat)
    onehotlabels = oh_enc.transform(dataset_cat)
    cat_array = onehotlabels.toarray()
    dataset_no_cat = dataset.select_dtypes(exclude=[object])
    non_cat_array = dataset_no_cat.as_matrix()
    return np.concatenate((non_cat_array,cat_array),axis = 1)
    

In [63]:
one_hot_encode(dataset)

array([[  1.        ,   1.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  1.        ,   1.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  1.        ,   1.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       ..., 
       [ 15.        ,  21.        ,   0.19857244, ...,   0.        ,
          0.        ,   0.        ],
       [ 15.        ,  21.        ,   0.19957007, ...,   0.        ,
          0.        ,   0.        ],
       [ 15.        ,  21.        ,   0.20120052, ...,   0.        ,
          0.        ,   0.        ]])